In [1]:
# init
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import yasa
import numpy as np
#import matplotlib.pyplot as plt

#X_train = h5py.File("X_train.h5", "r")
y_train = pd.read_csv("y_train.csv")

In [2]:
# Features
def deltabeta(data):
    N = len(data)
    sf = 125
    res = np.zeros((N, 2))
    for i in range(N):
        band = yasa.bandpower(data[i, 11:], sf, kwargs_welch={})
        d = band["Delta"]["CHAN001"]
        b = band["Beta"]["CHAN001"]
        res[i][0] = d
        res[i][1] = d / b
    return res

def extract_features(h5):
    data = h5["features"][:]
    features = []
    features.append(data[:, :])
    #features.append(data[:, :11])
    features.append(data[:, 11:].max(1).reshape(-1, 1))
    features.append(data[:, 11:].min(1).reshape(-1, 1))
    features.append(np.abs(data[:, 11:]).mean(1).reshape(-1, 1))
    features.append(deltabeta(data))
    ##features.append(nbmax(data[:, 11:]).reshape(-1, 1))
    features = np.concatenate(features, 1)
    return features

In [3]:
allfeatures = np.load("features_train_all.npy")

In [4]:
#features_train = np.load("features_train.npy")
y_train = pd.read_csv("y_train.csv")

In [5]:
# Train
#features_train = extract_features(X_train)
# Données
#X_train,X_test,y_train,y_test=train_test_split(allfeatures,y_train["label"])

In [6]:
y_train = np.squeeze(y_train["label"].values)

In [7]:
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

n = 466
clf = lgb.LGBMClassifier(n_estimators=n,num_leaves=10,learning_rate=0.07,
                         max_depth=10, random_state=42,verbose=1)
#clf.fit(X_train, y_train)
clf.fit(allfeatures, y_train)
#y_pred=clf.predict(X_test)
#matches=[int(y_pred[k]==np.array(y_test)[k]) for k in range(len(y_pred))]
#print("n_estimators =", n, "\naccuracy =", np.sum(matches)/len(y_pred))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.07, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=466, n_jobs=-1, num_leaves=10, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=1)

In [8]:
# test
X_test = h5py.File("X_test.h5", "r")
features_test = extract_features(X_test)
y_pred = clf.predict(features_test)
with open("y_benchmark.csv", "w") as f:
    f.write("".join(["id,label\n"] + ["{},{}\n".format(i, y) for i, y in enumerate(y_pred)]))

In [9]:
np.save("features_test", features_test)